## COVID JATIM

- Request

In [ ]:
import requests
import pandas as pd
url = "https://radarcovid19.jatimprov.go.id/"
req = requests.get(url)
html_string = req.text

html_string

- Data Rumah Sakit

In [ ]:
rs = html_string.split("var datarumahsakit=")
rs = rs[1].split(";")

rs_df = pd.read_json(rs[0])

rs_df

- Data Observasi

In [ ]:
do = html_string.split("var dataobservasi=")
do = do[1].split("];")[0]
do += "]"

do_df = pd.read_json(do)
do_df

- Positive

In [ ]:
positive = html_string.split("var datapositiflatlon=")
positive = positive[1].split(";")[0]

positive_df = pd.read_json(positive)

positive_df

In [ ]:
#Minify
rss = rs_df.copy()
rss = rss.rename(columns={
    'u': 'terakhir_update', 
    'instansi_rs': 'n', 
    'kontak': 'k',
    'iso_tanpa_vent': 'iso',
    'iso_vent': 'isov',
    'iso_tanpa_negatif': 'isotn',
    'lat': 'lat',
    'lon': 'lng',
    'kab_kota': 'kab',
    'ketersediaan_vrs': 'vrs',
    'ketersediaan_tvrs': 'tvrs',
    'ketersediaan_brs': 'brs'
})

rss

In [ ]:
dos = do_df.copy()
dos = dos.drop(["id", "created_at", "updated_at", "deleted_at"], axis=1)
dos = dos.rename(columns={
    'kabko': 'kab',
    'kecamatan': 'kec',
    'desa_kel': 'kel',
    'nama_tempat': 'n',
    'alamat': 'a',
    'daya_tampung': 'dt',
    'koordinator': 'koor',
    'nohp': 'hp',
    'dokumen': 'dok',
    'keterangan': 'ket',
    'lat': 'lat',
    'lon': 'lng'
})

dos

In [ ]:

positives = positive_df.copy()
positives = positives.drop("status", axis=1)
positives = positives.rename(columns={
    'umur': 'u',
    'kelamin': 'k',
    'kabkota': 'kab',
    'kecamatan': 'kec',
    'desa': 'kel',
    'lat': 'lat',
    'lon': 'lng',
    'status_pasien': 'sp'
})

positives


In [ ]:
#Rename latest json to yesterday suffix
#export new json to latest suffix
import os
import datetime

now = datetime.datetime.now()
now_suffix = now.strftime('%Y-%m-%d')
yesterday = now - datetime.timedelta(days = 1)
yesterday_suffix = yesterday.strftime('%Y-%m-%d')

try:
    os.rename('rs_latest.json', 'rs_' + yesterday_suffix + '.json')
    os.rename('do_latest.json', 'do_' + yesterday_suffix + '.json')
    os.rename('pos_latest.json', 'pos_' + yesterday_suffix + '.json')
except:
    pass


new_rss = rss.to_json(orient="records")
new_rss = '{"date": "' + now_suffix + '", "data":' + new_rss + '}'
rss_file = open("rs_latest.json", "w")
rss_file.write(new_rss)
rss_file.close()


new_dos = dos.to_json(orient="records")
new_dos = '{"date": "' + now_suffix + '", "data":' + new_dos + '}'
dos_file = open("do_latest.json", "w")
dos_file.write(new_dos)
dos_file.close()

new_pos = positives.to_json(orient="records")
new_pos = '{"date": "' + now_suffix + '", "data":' + new_pos + '}'
pos_file = open("pos_latest.json", "w")
pos_file.write(new_pos)
pos_file.close()